In [1]:
using MPSGE

Build the simple TWOBYTWO model:

In [2]:
m = Model()

endow = add!(m, Parameter(:endow, 1.))

add!(m, Sector(:X))
add!(m, Sector(:Y))
add!(m, Sector(:U))

add!(m, Commodity(:PX))
add!(m, Commodity(:PY))
add!(m, Commodity(:PU))
add!(m, Commodity(:PL))
add!(m, Commodity(:PK))

add!(m, Consumer(:RA, benchmark=150.))

add!(m, Production(:X, 1, :PX, 100, [Input(:PL, 50), Input(:PK, 50)]))
add!(m, Production(:Y, 1, :PY, 50, [Input(:PL, 20), Input(:PK, 30)]))
add!(m, Production(:U, 1, :PU, 150, [Input(:PX, 100), Input(:PY, 50)]))

add!(m, Demand(:RA, :PU, [Endowment(:PL, :(70 * $endow)), Endowment(:PK, 80.)]))

MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)


Next try to solve it with an iteration limit of 0 to replicate the benchmark:

In [3]:
solve!(m, cumulative_iteration_limit=0)

┌ Warning: Disabling output because PATH.jl does not support writing to this `stdout`. Are you using Atom-Hydrogen or Jupyter? See: https://github.com/odow/PATH.jl/issues/10.
└ @ PATHSolver C:\Users\david\.julia\packages\PATHSolver\fB6ZO\src\C_API.jl:517


MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Solution:
  X:	1.0
  Y:	1.0
  U:	1.0
  PX:	1.0
  PY:	1.0
  PU:	1.0
  PL:	1.0
  PK:	1.0
  PLPX:	50.0
  PKPX:	50.0
  PLPY:	20.0
  PKPY:	30.0
  PXPU:	100.0
  PYPU:	50.0
  RA:	150.0


Now change the value of the parameter `endow` and solve the model again:

In [4]:
set_value(endow, 1.1)
solve!(m)

┌ Warning: Disabling output because PATH.jl does not support writing to this `stdout`. Are you using Atom-Hydrogen or Jupyter? See: https://github.com/odow/PATH.jl/issues/10.
└ @ PATHSolver C:\Users\david\.julia\packages\PATHSolver\fB6ZO\src\C_API.jl:517


MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Solution:
  X:	1.0488088481734958
  Y:	1.03886011825708
  U:	1.0454820635822561
  PX:	1.0488566170769114
  PY:	1.0589010793419213
  PU:	1.0521941397098173
  PL:	1.0000455456233646
  PK:	1.1000501007115724
  PLPX:	52.44044240818329
  PKPX:	47.67312946227611
  PLPY:	21.17705705826264
  PKPY:	28.877805079697787
  PXPU:	100.31820580255854
  PYPU:	49.68330660298154
  RA:	165.00751506992486


Display an algebraic version of the model:

In [5]:
jm = algebraic_version(m)

Mixed complementarity problem with 15 constraints:
  (50.0 * (PL ^ 0.5 * PK ^ 0.5)) / PL - PLPX                                 ┴  PLPX
  (50.0 * (PL ^ 0.5 * PK ^ 0.5)) / PK - PKPX                                 ┴  PKPX
  (20.0 * (PL ^ 0.4 * PK ^ 0.6)) / PL - PLPY                                 ┴  PLPY
  (30.0 * (PL ^ 0.4 * PK ^ 0.6)) / PK - PKPY                                 ┴  PKPY
  (100.0 * (PX ^ 0.6666666666666666 * PY ^ 0.3333333333333333)) / PX - PXPU  ┴  PXPU
  (50.0 * (PX ^ 0.6666666666666666 * PY ^ 0.3333333333333333)) / PY - PYPU   ┴  PYPU
  (PL * PLPX + PK * PKPX) - 100.0 * PX                                       ┴  0.0 < X
  (PL * PLPY + PK * PKPY) - 50.0 * PY                                        ┴  0.0 < Y
  (PX * PXPU + PY * PYPU) - 150.0 * PU                                       ┴  0.0 < U
  (+(PXPU * U) + 0.0) - 100.0 * X                                            ┴  0.001 < PX
  (+(PYPU * U) + 0.0) - 50.0 * Y                                             ┴  0.00